In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 59.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=681963627b841eb8809cebc0830e38d58ad51774ad6c18f4ee4992656c91ba3c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("SparkStreaming").getOrCreate()

In [ ]:
tweets=[
    ["Estou muito animado com esse projeto! Esse sim vai m... https://t.co/u4EGCRvwPb"],
    ["@danpsiqueira @Rodrigo A área de engenharia de dados só vai crescer"],
    ["RT @Rodrigotechlead: Eu estou preparando uma formação inédita para todos que queiram começar na área"],
    ["RT @Davidneves: O meu objetivo é converter o maior número de bolas de 3 no jogo"],
    ["RT @RomuloSM: https://t.co/yuoLpFEboa"],
    ["@Renato @David Sucesso total que fala..."],
    ["RT @MariaDB: Meus gatos são muito lindos, e eu posso provar"],
    ["RT @beatriz2801: Não vejo a hora de produzir o curso de Dax avançado...."],
    ["@Afonso Vou fazer um dashboard do @Flamengo  💪❤️🖤\n#CRF #VamosFlamengo https://t.co/6Uhokyh6YL"],
    ["RT @Libertadores: 🙌 ¡Los equipos ya clasificados a los octavos de final de la CONMEBOL #Libertadores!\n🇦🇷 @EdelpOficial y @CATalleresdecba…"],
    ["RT @MillenaG: @Igor Deixa eu te falar então...\nSim. Os engenheiros do açaí vem com tudo"],
]

In [ ]:
tweets = spark.createDataFrame(tweets, ['tweets'])

In [ ]:
tweets.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+
|tweets                                                                                                                                        |
+----------------------------------------------------------------------------------------------------------------------------------------------+
|Estou muito animado com esse projeto! Esse sim vai m... https://t.co/u4EGCRvwPb                                                               |
|@danpsiqueira @Rodrigo A área de engenharia de dados só vai crescer                                                                           |
|RT @Rodrigotechlead: Eu estou preparando uma formação inédita para todos que queiram começar na área                                          |
|RT @Davidneves: O meu objetivo é converter o maior número de bolas de 3 no jogo                                                  

In [ ]:
type(tweets)

pyspark.sql.dataframe.DataFrame

In [ ]:
tweets\
    .select(f.split('tweets', " ").alias('word'))\
    .show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|word                                                                                                                                                              |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Estou, muito, animado, com, esse, projeto!, Esse, sim, vai, m..., https://t.co/u4EGCRvwPb]                                                                       |
|[@danpsiqueira, @Rodrigo, A, área, de, engenharia, de, dados, só, vai, crescer]                                                                                   |
|[RT, @Rodrigotechlead:, Eu, estou, preparando, uma, formação, inédita, para, todos, que, queiram, começar, na, área]                                              |
|[RT, @Dav

In [ ]:
tweets\
    .select(f.explode(f.split('tweets', " ")).alias('word'))\
    .show(50, truncate=False)

+-----------------------+
|word                   |
+-----------------------+
|Estou                  |
|muito                  |
|animado                |
|com                    |
|esse                   |
|projeto!               |
|Esse                   |
|sim                    |
|vai                    |
|m...                   |
|https://t.co/u4EGCRvwPb|
|@danpsiqueira          |
|@Rodrigo               |
|A                      |
|área                   |
|de                     |
|engenharia             |
|de                     |
|dados                  |
|só                     |
|vai                    |
|crescer                |
|RT                     |
|@Rodrigotechlead:      |
|Eu                     |
|estou                  |
|preparando             |
|uma                    |
|formação               |
|inédita                |
|para                   |
|todos                  |
|que                    |
|queiram                |
|começar                |
|na         

In [ ]:
tweets\
    .select(f.explode(f.split('tweets', " ")).alias('word'))\
    .withColumn('word', f.regexp_replace('word', r'http\S+', ''))\
    .withColumn('word', f.regexp_replace('word', '@\w+', ''))\
    .withColumn('word', f.regexp_replace('word', '#', ''))\
    .withColumn('word', f.regexp_replace('word', 'RT', ''))\
    .withColumn('word', f.regexp_replace('word', ':', ''))\
    .na.replace('', None)\
    .na.drop()\
    .show(50, truncate=False)

+----------+
|word      |
+----------+
|Estou     |
|muito     |
|animado   |
|com       |
|esse      |
|projeto!  |
|Esse      |
|sim       |
|vai       |
|m...      |
|A         |
|área      |
|de        |
|engenharia|
|de        |
|dados     |
|só        |
|vai       |
|crescer   |
|Eu        |
|estou     |
|preparando|
|uma       |
|formação  |
|inédita   |
|para      |
|todos     |
|que       |
|queiram   |
|começar   |
|na        |
|área      |
|O         |
|meu       |
|objetivo  |
|é         |
|converter |
|o         |
|maior     |
|número    |
|de        |
|bolas     |
|de        |
|3         |
|no        |
|jogo      |
|Sucesso   |
|total     |
|que       |
|fala...   |
+----------+
only showing top 50 rows

